In [ ]:
import json
import requests
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings('ignore')
from ast import literal_eval
from pandas import json_normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# api로 불러온 파일을 json 형태로 저장 

API_KEY = 'c9770fb1bb1a8093a00403d97f04494a'
BASE_URL = 'https://api.themoviedb.org/3/movie/popular/'
movie_list = []

for i in range(1,251):
    response = requests.get(BASE_URL, { 'api_key': API_KEY, 'page': i })
    json_data = response.json()


    for data in json_data.get('results'):
        # poster_path가 null 값인 movie는 저장하지 않음!
        if data.get('poster_path'):
            movie = {
                'model': 'movies.movie',
                'fields':{
                    'title': data.get('title'),
                    'id': data.get('id'),
                    'overview': data.get('overview'),
                    'poster_path': data.get('poster_path'),
                    # 
                    'genres': data.get('genre_ids'),
                    'vote_average': data.get('vote_average'),
                    'vote_count': data.get('vote_count')
                }
            }
            movie_list.append(movie)

with open('movies_poster2.json', 'w', encoding='utf-8') as f:
    json.dump(movie_list, f)

In [ ]:
# json 파일을 dataframe 으로 변환
def from_json_to_df(jsfile):
    json1 = pd.json_normalize(jsfile['fields'])
    return json1

# dataframe 틀 잡기 
f = open('movies_poster2.json')
movie_poster2 = json.load(f)
movie_poster = movie_poster2[0]
df = pd.json_normalize(movie_poster['fields'])

for movie in map(from_json_to_df, movie_poster2):
    df = df.append(movie, ignore_index=True)

df = df[1:].reset_index(drop=True)
 

In [ ]:
# 알고리즘 적용하기

# 영화정보를 벡터로 만들기
movies_df = df.copy()
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(map(str, x)))
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(movies_df['genres_literal'])

# 벡터 간 유사도 구하기
genre_sim = cosine_similarity(genre_mat, genre_mat)
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] 

In [ ]:
# weighted_vote 구하기 : 같은 점수라면 더 많은 투표를 얻은 영화를 선택한다

percentile = 0.6
m = movies_df['vote_count'].quantile(percentile)
C = movies_df['vote_average'].mean()

def weighted_vote_average(record):
    v = record['vote_count']
    R = record['vote_average']
    
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )   

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [ ]:
def find_sim_movie(df, sorted_ind, movie_id, top_n=10):
    # df에서 id인 행과 그 인덱스
    title_movie = df[df['id'] == movie_id]
    title_index = title_movie.index.values
    
    # top_n의 2배에 해당하는 쟝르 유사성이 높은 index 추출 - 기준영화
    similar_indexes = sorted_ind[title_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출 
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]


In [ ]:
# movies와 merge할 df_recom 생성

df_recom = pd.DataFrame()

for id in movies_df['id']:
    similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, id, 3)
    
    recommends = []
    for index, row in similar_movies.iterrows():
        recommends.append(row['id'])
    
    df_add = pd.DataFrame(data={'id':id, 'recommends':[recommends]})
    df_recom = pd.concat([df_recom, df_add])

df_final = movies_df.merge(df_recom)
df_final.drop(columns=['genres_literal'], inplace=True)

In [ ]:
# dataframe을 다시 json 으로 변환

movie_list = []

for idx, data in df_json.iterrows():
    movie = {
        'model': 'movies.movie',
        'fields':{
            'title': data.get('title'),
            'id': data.get('id'),
            'overview': data.get('overview'),
            'poster_path': data.get('poster_path'),
            # 
            'genres': data.get('genres'),
            'vote_average': data.get('vote_average'),
            'vote_count': data.get('vote_count'),
            'weighted_vote': data.get('weighted_vote'),
            'recommends': data.get('recommends')
        }
    }
    movie_list.append(movie)

with open('movies_recommends.json', 'w', encoding='utf-8') as f:
    json.dump(movie_list, f)